To achieve machine unlearning without retraining on the filtered dataset, we can modify the weights of the neural network model directly to forget about class 2. One way to do this is by adjusting the weights corresponding to class 2 to be closer to the weights corresponding to class 0 (or any other class we want the model to learn to resemble).

In [19]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

# Load MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Normalize pixel values
X_train = X_train / 255.0
X_test = X_test / 255.0

# Convert labels to one-hot encoding
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

# Define and train a model on all classes
model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Accuracy on test data before unlearning class 2:", accuracy)
from sklearn.metrics import confusion_matrix
y_pred = model.predict(X_test)
conf_matrix = confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))
print("Confusion matrix:")
print(conf_matrix)

# Define the class to forget
class_to_forget = 2

# Modify the weights of the model to forget about class 2
weights = model.layers[-1].get_weights()
weights[0][:,class_to_forget].fill(0) # Replace the weights corresponding to class 2 with zeros
weights[1][class_to_forget] =0 # Replace the bias corresponding to class 2 with zero
model.layers[-1].set_weights(weights)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Accuracy on test data after unlearning class 2:", accuracy)
y_pred = model.predict(X_test)
conf_matrix = confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))
print("Confusion matrix:")
print(conf_matrix)

Epoch 1/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.2559 - accuracy: 0.9269 - val_loss: 0.1351 - val_accuracy: 0.9603
Epoch 2/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1104 - accuracy: 0.9674 - val_loss: 0.0967 - val_accuracy: 0.9705
Epoch 3/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0766 - accuracy: 0.9770 - val_loss: 0.0870 - val_accuracy: 0.9734
Epoch 4/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0582 - accuracy: 0.9822 - val_loss: 0.0792 - val_accuracy: 0.9744
Epoch 5/5
313/313 [==============================] - 0s 951us/step - loss: 0.0713 - accuracy: 0.9780
Accuracy on test data before unlearning class 2: 0.9779999852180481
313/313 [==============================] - 0s 1ms/step
Confusion matrix:
[[ 972    0    1    1    1    1    2    1    1    0]
 [   1 1124    2    0    0    1    2    1    4    0]
 [  10    3 1010    1    0    0    2    4    2    0]
 [   2    0    3  998    0 

In this code:
1. We define and train a neural network model on all classes of the MNIST dataset.
2. We identify the class we want to forget, which in this case is class 2.
3. We modify the weights of the output layer corresponding to class 2 to be zeros, effectively "forgetting" about class 2.
4. We evaluate the accuracy of the modified model on the test data to see how well it performs after unlearning class 2.

Although it worked. but I don't think it is the correct mechansim because zeroing the weights for the forget class is only applied on theoutput layer. What about rest of the hidden layers of the network, it still has the weights corresponding to the forget class.

I am thinking in terms of the progressive learning setup. 

#### To perform weight pruning to minimize the influence of the forgotten class in all layers of the network, we can iterate through each layer of the model and adjust the weights accordingly. We will set the weights corresponding to the forgotten class to zero or reduce their magnitude.

We can incorporate L1 or L2 regularization to encourage sparsity in the weights of the neural network, effectively reducing the influence of the specific class we want to forget. We use L2 in the below code.

Here's how you can implement this in Python using TensorFlow/Keras:

In [26]:
# Define and train a model on all classes
model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    Dense(10, activation='softmax')
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Accuracy on test data before unlearning class 2:", accuracy)
from sklearn.metrics import confusion_matrix
y_pred = model.predict(X_test)
conf_matrix = confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))
print("Confusion matrix:")
print(conf_matrix)


# Define the class to forget
class_to_forget = 2

# Set L2 regularization penalty for weights corresponding to the forgotten class
for layer in model.layers:
    if isinstance(layer, Dense):
        weights, biases = layer.get_weights()
        if layer == model.layers[-1]:
            weights[:, class_to_forget] = 0  # Zero out weights for the forgotten class in the output layer
        else:
            weights[:, class_to_forget] *= 0.01  # Apply L2 regularization penalty to weights in hidden layers
        layer.set_weights([weights, biases])
'''
# Now do it at the output layer
weights = model.layers[-1].get_weights()
weights[0][:,class_to_forget].fill(0) # Replace the weights corresponding to class 2 with zeros
weights[1][class_to_forget] =0 # Replace the bias corresponding to class 2 with zero
model.layers[-1].set_weights(weights)
'''

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Accuracy on test data after weight pruning:", accuracy)

# Compute confusion matrix
y_pred = model.predict(X_test)
conf_matrix = confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))
print("Confusion matrix:")
print(conf_matrix)

Epoch 1/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.6044 - accuracy: 0.8999 - val_loss: 0.4036 - val_accuracy: 0.9267
Epoch 2/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3972 - accuracy: 0.9281 - val_loss: 0.3785 - val_accuracy: 0.9340
Epoch 3/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3602 - accuracy: 0.9375 - val_loss: 0.3616 - val_accuracy: 0.9357
Epoch 4/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3357 - accuracy: 0.9416 - val_loss: 0.3200 - val_accuracy: 0.9471
Epoch 5/5
313/313 [==============================] - 0s 1ms/step - loss: 0.3316 - accuracy: 0.9388
Accuracy on test data before unlearning class 2: 0.9387999773025513
313/313 [==============================] - 0s 1ms/step
Confusion matrix:
[[ 953    0    0    3    0   11    5    2    3    3]
 [   0 1128    0    3    0    0    1    0    3    0]
 [  15    7  890   47    5    1    7   12   38   10]
 [   0    1    0  992    0   

#### Sensitivity analysis for finding important weights.

We define a neural network model with two hidden layers and an output layer.
We train the model on the MNIST dataset.
For each layer (excluding the output layer), we perturb a random sample of weights and measure the change in accuracy on the specified class.
We print the top 10 most important weights for each layer based on sensitivity.

We exclude the output layer because it is well known that the weights directed towards the forgetclass node are most important for that class.

In [103]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

# Load the MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Normalize pixel values to range [0, 1]
X_train, X_test = X_train / 255.0, X_test / 255.0

# Define the class to analyze
class_to_analyze = 2

# Filter the dataset for the specified class
X_train_class = X_train[y_train == class_to_analyze]
X_test_class = X_test[y_test == class_to_analyze]

# Define the neural network model
model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(128, activation='relu'),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer=Adam(), loss=SparseCategoricalCrossentropy(), metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))

# Calculate the baseline accuracy
baseline_loss, baseline_accuracy = model.evaluate(X_test, y_test)
print("Baseline accuracy:", baseline_accuracy)

y_pred_prob = model.predict(X_test)
y_pred = np.argmax(y_pred_prob, axis=1)

# Compute confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

print("Confusion Matrix:")
print(conf_matrix)

Epoch 1/5
1875/1875 [==============================] - 15s 8ms/step - loss: 0.2345 - accuracy: 0.9305 - val_loss: 0.1173 - val_accuracy: 0.9653
Epoch 2/5
1875/1875 [==============================] - 15s 8ms/step - loss: 0.0991 - accuracy: 0.9697 - val_loss: 0.0997 - val_accuracy: 0.9669
Epoch 3/5
1875/1875 [==============================] - 15s 8ms/step - loss: 0.0694 - accuracy: 0.9782 - val_loss: 0.0857 - val_accuracy: 0.9736
Epoch 4/5
1875/1875 [==============================] - 15s 8ms/step - loss: 0.0531 - accuracy: 0.9829 - val_loss: 0.0722 - val_accuracy: 0.9783
Epoch 5/5
313/313 [==============================] - 1s 3ms/step - loss: 0.0821 - accuracy: 0.9757
Baseline accuracy: 0.9757000207901001
313/313 [==============================] - 1s 3ms/step
Confusion Matrix:
[[ 971    0    1    1    1    0    2    2    2    0]
 [   0 1126    0    3    0    1    2    0    3    0]
 [   3    1  997   11    1    0    2    4   12    1]
 [   1    0    5  982    0    7    0    5    0   10]
 [

In [88]:
# Define the fraction of weights to perturb
fraction_to_perturb = 0.01  # 1% of the weights will be perturbed

# Iterate through each layer and calculate sensitivity
sensitivities = {}
weights_location = {}
for k,layer in enumerate(model.layers[:-1]):  # Exclude output layer
    if hasattr(layer, 'weights') and len(layer.weights) > 0:
        weights, biases = layer.get_weights()
        num_weights_to_perturb = int(fraction_to_perturb * weights.size)
        weight_indices_to_perturb = np.random.choice(weights.size, num_weights_to_perturb, replace=False)
        sensitivities[k+1] = []
        weights_location[k+1] = []
        for idx in weight_indices_to_perturb:
            i, j = np.unravel_index(idx, weights.shape)
            # Temporarily set the weight to zero
            original_weight = weights[i, j].copy()
            weights[i, j] = 0

            # Set the modified weights to the model
            layer.set_weights([weights, biases])

            # Evaluate the model with the modified weights
            loss, accuracy = model.evaluate(X_test_class, np.full(len(X_test_class), class_to_analyze), verbose=0)

            # Restore the original weight
            weights[i, j] = original_weight

            # Calculate the change in accuracy
            sensitivity = baseline_accuracy - accuracy
            sensitivities[k+1].append(sensitivity)
            weights_location[k+1].append((i,j))
        # Sort the sensitivities and get the indices of the most important weights
        important_weight_indices = np.argsort(sensitivities[k+1])
        print(f"Layer: {layer.name}")
        print("Top 10 most important weights:")
        print("Weight index:", np.array(weights_location[k+1])[important_weight_indices][:10] , "Sensitivity:", np.array(sensitivities[k+1])[important_weight_indices][:10])

Layer: dense_58
Top 10 most important weights:
Weight index: [[205  14]
 [693  83]
 [650  27]
 [654  70]
 [736   3]
 [181  72]
 [487  20]
 [ 33  16]
 [ 53  11]
 [663 127]] Sensitivity: [-0.00027519 -0.00027519 -0.00027519 -0.00027519 -0.00027519 -0.00027519
 -0.00027519 -0.00027519 -0.00027519 -0.00027519]
Layer: dense_59
Top 10 most important weights:
Weight index: [[ 12  71]
 [ 26  39]
 [105  35]
 [103 123]
 [ 58   3]
 [ 81  98]
 [114 122]
 [  3  75]
 [ 37  46]
 [ 53   1]] Sensitivity: [-0.00027519 -0.00027519 -0.00027519 -0.00027519 -0.00027519 -0.00027519
 -0.00027519 -0.00027519 -0.00027519 -0.00027519]


In [102]:
#Zero out the sensitive weights
for k,layer in enumerate(model.layers):  # Exclude output layer
    if isinstance(layer, Dense):
        weights, biases = layer.get_weights()
        if layer == model.layers[-1]:
            weights[:, class_to_forget] = 0  # Zero out weights for the forgotten class in the output layer
        else:
            for (i,j) in weights_location[k+1][:30]: #zeroing 30 most sensitive weights
                weights[i,j] = 0 #zero out sensitive weights
        layer.set_weights([weights, biases])
        
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Accuracy on test data after weight pruning:", accuracy)

y_pred_prob = model.predict(X_test)
y_pred = np.argmax(y_pred_prob, axis=1)

# Compute confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

print("Confusion Matrix:")
print(conf_matrix)

313/313 [==============================] - 1s 4ms/step - loss: 0.4205 - accuracy: 0.8829
Accuracy on test data after weight pruning: 0.8828999996185303
313/313 [==============================] - 1s 3ms/step
Confusion Matrix:
[[ 970    1    0    2    0    1    2    1    3    0]
 [   0 1115    0    6    0    1    2    2    9    0]
 [  38  218   63  313   35    0   38  311   15    1]
 [   2    0    0  996    0    2    0    5    1    4]
 [   0    1    1    1  960    1    7    1    2    8]
 [   2    0    0    6    0  876    4    0    3    1]
 [   4    2    0    2    4    4  942    0    0    0]
 [   1    0    0    9    0    1    0 1007    5    5]
 [  11    1    0   11    3    9    5    2  931    1]
 [   5    2    0    5    9   10    1    4    4  969]]


#### This is showing some improvements than the regularization or the zeroing of output layer-only weights.